# LSST DRP Computational Requirements

Created September 2015 to satisfy the requirements of the DRP Computational Budget key performance metric (DR1; see DLP-314, DM-3083, DM-3897). See also LDM-138, LSE-81. This is based on a Python script provided by K-T Lim.

In [1]:
# Disable ShimWarnings caused by AstroPy assuming an older version of IPython
from IPython.utils.shimmodule import ShimWarning
import warnings
warnings.simplefilter("ignore", category=ShimWarning)
from astropy import units as u
from math import pi

We ultimately need to end up with a value in TFLOPS (ie, Tera FLating point Operations Per Second). We make measurements of execution time on particular computers, of known clock speed (cycles per second), and convert them to FLOPS as required.

In [2]:
flop = u.def_unit('FLOP') # FLoating point OPeration
cycle = u.def_unit('cycle', 4*0.68*flop) # Conversion rate based on average efficiency of 2011 TOP500
tflops = u.def_unit('TFLOPS', (flop*10**12)/(1*u.second))

The `Telescope` class describes the instrument we are using to observe. We have only one: LSST itself.

In [3]:
class Telescope(object):
    def __init__(self, nCcds, nFilters, fieldOfView):
        self.nCcds = nCcds
        self.nFilters = nFilters
        self.fieldOfView = fieldOfView
    
lsst = Telescope(189, 6, pi*(1.75*u.degree)**2)

The `Survey` class describes the survey being carried out -- including duration, number of objects observerd, etc. For this exercise, we are using the six months of the LSST wide & fast survey.

In [4]:
class Survey(object):
    def __init__(self, duration, visits, epochs, fields, stars, surveyStars, forcedSources, computeTime):
        self.duration = duration            # Survey duration
        self.nVisits = visits               # Total number of visits in the survey
        self.nEpochs = epochs               # Number of visits to any given object
        self.nFields = fields               # Survey area expressed in terms of fields-of-view
        self.nStars = stars                 # Total number of stars
        self.nOutOfPlaneStars = surveyStars # Stars observed out of the galactic plane
        self.nForcedSources = forcedSources # Total number of forced source measurements
        self.computeTime = computeTime

# The relevant data release for the key performance metric is DR2, which takes place after 1 year.
lsstWideAndFast1Year = Survey(u.year,
                              2750000 / 10, # OpSim + 10% margin
                              1056 / 10,
                              2604,         # Science Requirements
                              1.71E+10,     # LSE-81 D56
                              5.63E+09,     # LSE-81 E56
                              2.91E+12,     # LSE-81 H56  
                              9*u.week)     # This production runs in 6 months = 9 weeks processing

The `Computer` class describes the various computers we have used to make measurements in terms of their clock speed.

In [5]:
class Computer(object):
    def __init__(self, clockSpeed):
        self.clockSpeed = clockSpeed

abe = Computer(2.33e9 * cycle / u.second)      # Machine used for PT 1.1 processing
lsst_dev = Computer(3.1e9 * cycle / u.second)  # lsst-dev.ncsa.illinois.edu
tiger3 = Computer(2.8e9 * cycle / u.second)    # tiger3.princeton.edu
ksk_mbp = Computer(3e9 * cycle / u.second)     # Macbook Pro used for difference imaging tests at Bremerton

Next, we define the scientific and algorithmic approaches being taken.

In [6]:
class CoaddParameters(object):
    def __init__(self, pixelDensityFactor, imageReuse, deblendFactor):
        # Increased pixel density factor for co-adds
        self.pixelDensityFactor = pixelDensityFactor
        # Number of times each image is reused in making deep coadds
        self.imageReuse = imageReuse
        # Deblend factor: extra coadd measurement time due to deblending
        self.deblendFactor = deblendFactor
        
coaddParams = CoaddParameters(1,   # LSE-81 G157 suggests 2; Jim Bosch suggests this is outdated
                              4,   # Per LSE-81 G229
                              0.1) # LDM-138 otherInput_2 B21; no obvious justification

class TemplateParameters(object):
    def __init__(self, templateTypes, templateVersions, templateDepth):
        # Number of different types of templates to be prepared
        self.nTypes = templateTypes
        # Different versions corresponding to e.g. differing airmass
        self.nVersions = templateVersions
        # Number of images being coadded for each template
        self.depth = templateDepth
        
templateParams = TemplateParameters(6, # One per filter
                                    3, # Per LSE-81 G166
                                    5) # Per LSE-81 G228

## Stack processing

We are now ready to quantify the various processing stages the stack goes through. Here, following LDM-138, we include difference imaging, generation of coadds (both deep coadds for measurement and templates for differencing), detection and measurement of sources and coadds, and multi-epoch object characterization. We do *not* include single frame measurement or astro- or photo-metric relative calibration. (Presumably because they don't run at the same time on the same hardware, but it's not completely clear why.)

### Generate difference images

According to [DM-3274](https://jira.lsstcorp.org/browse/DM-3274), difference imaging (with no associated source measurement) was benchmarked as taking 35 seconds per CCD on 2k by 4k CCDs when run on `lsst-dev`. We expect this to scale linearly with pixel count to the 4k by 4k CCDs used by LSST since it is involves applying a kernel to each pixel. We therefore estimate the total time as 70 seconds per CCD.

In [7]:
def calcDiffimCycles(timePerCcd, telescope, survey, computer):
    return timePerCcd * telescope.nCcds * survey.nVisits * computer.clockSpeed

diffimCycles = calcDiffimCycles(70*u.second, lsst, lsstWideAndFast1Year, lsst_dev)
diffimCycles

<Quantity 1.1278575e+19 cycle>

### Constructing coadds

We construct two types of coadds: templates for difference imaging and deep coadds for measurement. The time to warp-and-add a CCD image to the coadd is the same for both, but the number and depth of the coadds created are different.

The warp-and-add step is contained within the `makeCoaddTempExp` and `assembleCoadd` tasks. We it using HSC engineering data on `tiger3`. Each coadd consisted of 60 calibrated exposures (`calexp`s) from HSC, each 2k by 4k pixels. We assume that the time scales linear with number of pixels to LSST's 4k by 4k CCDs.

In [8]:
nCalexps, hscToLsstScale = 60, 2
makeTempExpTime = u.second * ((190.72 - 10.17) + (189.47 - 10.24))/2 # Measurement repeated to check for consistency
assembleCoaddTime = u.second * ((49.31 - 10.24) + (49.22 - 9.73))/2
coaddCcdTime = hscToLsstScale * (makeTempExpTime + assembleCoaddTime) / nCalexps
coaddCcdTime

<Quantity 7.305666666666666 s>

However, this may not be a reliable estimate of the coaddition time in practice. Previous versions of this calculation have assumed that coaddition was similar to differencing imaging ("warp + arithmetic") and, as per above, the difference imaging calculation is a factor of ~10 larger. This is likely because differencing also involves PSF matching, an operation which may dominate the run time, and which is required for some times of coaddition but which is not included in the figure above.

### Generate difference templates

In [9]:
def calcTemplateCoadditionCycles(timePerCcd, coaddParams, templateParams,
                                 telescope, survey, computer):
    return (timePerCcd * telescope.nCcds * coaddParams.pixelDensityFactor * 
            survey.nFields * templateParams.nTypes * templateParams.nVersions *
            templateParams.depth * computer.clockSpeed)
    
templateCycles = calcTemplateCoadditionCycles(coaddCcdTime, coaddParams, templateParams,
                                              lsst, lsstWideAndFast1Year, tiger3)
templateCycles

<Quantity 9.060729763679999e+17 cycle>

### Generate deep coadds

The time taken to coadd CCDs was as the combined runtime of `makeCoaddTempExp` and `assembleCoadd` run on HSC engineering data on `tiger3.princeton.edu` using version `v11_0_rc2` of the stack.

In [10]:
def calcDeepCoadditionCycles(timePerCcd, coaddParams, telescope, survey, computer):
    # Note that each time an image is reused, it will need to be re-warped, so this effect is multiplicative
    return timePerCcd * telescope.nCcds * computer.clockSpeed * coaddParams.imageReuse * survey.nVisits

deepCoaddCycles = calcDeepCoadditionCycles(coaddCcdTime, coaddParams, lsst, lsstWideAndFast1Year, abe)
deepCoaddCycles

<Quantity 3.538916073e+18 cycle>

### Coadd source detection and measurement

Coadd source detection and measurement across multiple bands proceeds as follows:

* A source detection step is run on each band independently;
* The per-band source detection lists are merged to create a single master list;
* The sources in the master list are measured on each coadd independently.

Sources will be detected and measured on all coadds generated in the previous step. This will populate the catalog of `CoaddSources`.

We perform source detection & measurement on the HSC-I, -R and -Y bands of the coadded data. Each coadd has dimensions 4200 by 4200 pixels, making it slightly bigger than the 4000 by 4072 pixel CCDs.

Note that this procedure makes no attempt to correct for depth (and hence source numbers) achieved in HSC vs LSST.

In [11]:
coaddToCcd = (4000.0 * 4072.0) / (4200.0 * 4200.0)

# One time recorded per band
detectTimePerCcd = (8.405 + 7.986 + 8.347) * coaddToCcd * u.second / 3

# One time recorded overall
# Expect the merge time to vary quadratically with the number of catalogs;
# This test used 3, will ultimately have nFilters+1.
mergeTime = (lsst.nFilters + 1)**2 * 12.798 / (3**2) * u.second

# One time recorded per band
# Note that the measurement step includes deblending, so we do not add an extra "deblend factor"
measureTimePerCcd = (138.512 + 129.993 + 108.708) * coaddToCcd * u.second / 3


coaddSrcDetAndMeasCycles = (detectTimePerCcd + measureTimePerCcd) * lsst.nCcds * \
                           lsst_dev.clockSpeed * coaddParams.pixelDensityFactor * \
                           (lsst.nFilters + 1) * lsstWideAndFast1Year.nFields
coaddMergeCycles = mergeTime * lsst_dev.clockSpeed * lsstWideAndFast1Year.nFields
coaddProcessCycles = coaddMergeCycles + coaddSrcDetAndMeasCycles
coaddProcessCycles

<Quantity 1.32180807626448e+18 cycle>

### Multi-epoch object characterization

For each out-of-plane source, we both perform forced photometry on the CCD and multifit.

In Summer 2015, we do not have multifit capability in the stack. Instead, we estimate the performance as 0.112 seconds per source, as presented by Jim Bosch at the 2013 FDR.

We estimate the forced photometry time based on measurements of 10983 sources on HSC data using stack `v11_0_rc2`.

In [12]:
# Forced photometry:
# The same sources are measured in each image
numSources = 10983
# One time recorded per image
timePerObject = u.second * (62.428 + 62.082 + 56.435) / (3 * numSources)

# Multifit:
timePerObject += 200 * 0.56 / 1000 * u.second

def calcMultiEpochCycles(timePerObject, survey, computer):
    galacticPlaneObjects = survey.nStars - survey.nOutOfPlaneStars
    galacticPlaneForcedSrcs = galacticPlaneObjects * survey.nEpochs
    outOfPlaneForcedSrcs = survey.nForcedSources - galacticPlaneForcedSrcs
    return outOfPlaneForcedSrcs * timePerObject * computer.clockSpeed

multiEpochCycles = calcMultiEpochCycles(timePerObject, lsstWideAndFast1Year, lsst_dev)
multiEpochCycles

<Quantity 6.187323711471183e+20 cycle>

### Alert SQDA

SDQA Pipeline shall provide low-level data collection functionality for science data quality analysis of Level 1, 2, and Calibration Processing pipelines. It has been prototyped as `pipeQA`, and it's that that these results are based on. Note that these figures have not been updated as part of the Summer 2015 release.


In [13]:
def calcAlertSDQA(timePerVisit, survey, computer):
    return timePerVisit * survey.nVisits * computer.clockSpeed

sdqaCycles = calcAlertSDQA(600*u.second, lsstWideAndFast1Year, abe)
sdqaCycles

<Quantity 3.8445e+17 cycle>

## Totals

In [14]:
totalCycles = diffimCycles + templateCycles + deepCoaddCycles + coaddProcessCycles + multiEpochCycles + sdqaCycles
totalFlop = totalCycles.to(flop)
totalFlop

<Quantity 1.7303611657018826e+21 FLOP>

The required FLOPS of the compute hardware depends on the duration of the DRP run. The final total is:

In [15]:
(totalFlop / lsstWideAndFast1Year.computeTime).to(tflops)

<Quantity 317.89410010690085 TFLOPS>

Excluding any potential future algorithmic efficiency improvements.